# Setup

In [39]:
!pip install pytesseract pillow transformers azure-cognitiveservices-vision-computervision transformers torch python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.2 MB/s eta 0:00:00


In [40]:
# Import libraries
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import os
from PIL import Image
import io
import time
from docx import Document
from docx.shared import RGBColor

In [4]:
computervision_client = ComputerVisionClient(AZURE_ENDPOINT, CognitiveServicesCredentials(AZURE_KEY))

# Test OCR API

In [6]:
# Test OCR with an example image
image_path = "test_ocr.jpg"  # Replace with the path to your test image

def test_ocr(image_path):
    with open(image_path, "rb") as image_stream:
        # Call the API to analyze the image
        ocr_response = computervision_client.read_in_stream(image_stream, raw=True)

        # Get operation location (contains the operation ID)
        operation_location = ocr_response.headers["Operation-Location"]
        operation_id = operation_location.split("/")[-1]

        # Wait for the operation to complete
        while True:
            result = computervision_client.get_read_result(operation_id)
            if result.status not in [OperationStatusCodes.not_started, OperationStatusCodes.running]:
                break
            time.sleep(1)

        # Check the results
        if result.status == OperationStatusCodes.succeeded:
            read_results = result.analyze_result.read_results
            for page in read_results:
                for line in page.lines:
                    print(line.text)
        else:
            print("OCR operation failed.")

# Run the test
test_ocr(image_path)

Nasdaq & AMEX
Stocks in bold rose or fell 5% or more
USA
Track your investments with our continuously
TODAY
updated stocks. Visit us on the web at
.com
money.usatoday.com
52-week
52-week
High
Low
Stock
Last Change
High
Low Stock
Last Change
45.71 32.50
Biomet
-A-
36.71 -0.42
2.76
1.20
Biomira
1.46 +0.03
9.07
5.13
BloScrip
8.05 +0.34
9.19
6.89 ABX Air n
7.52 -0.10
68.88 50.65
Biosite
50.05 -4.57
33.25 12.40
ACMoore
13.58 -1.57
212.25 131.03
BiotechT
204.66 -0.84
31.38
13.51
ADA-ES
20.96 +3.16
8.50
1.40
BirchMt gn
6.52
-0.45
27.14
12.88
ADC Tel rs
23.21
+0.13
18.21
10.73
Bickbaud
17.90 +0.70
30.40
16.70
ADECD
27.32 +0.73
52.73
13.86
BluCoat
41.29 +1.30
16.45
10.47
AFC Ent s
15.40 -0.14
44.35
24.15
BlueNile
40.30
-1.10
8.37
4.50
ASE Tst
7.76 +0.40
26.45
19.91
BobEvn
22.99
19.25
12.75
ASM Intl
17.65 -0.03
15.94
6.12
Bodisen n
15.45
+0.45
20.92
13.94
ASML Hid
21.24 +0.46
6.21
1.56
Bookham
5.94 +0.06
27.38
16.39
ASV Inc 5
26.76 +0.14
11.80
4.99
Borland
6.68 +0.14
19.82
10.47
ATI Tech
17.89
+

# Test Gen AI model

In [25]:
# Load the pre-trained model and tokenizer
model_name = "gpt2"  # You can use "EleutherAI/gpt-neo-125M" or another model
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Test the model with a simple prompt
def generate_text(prompt):
    # Tokenize the input prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt")

    # Generate text based on the prompt
    outputs = model.generate(
        inputs,
        max_length=100,  # Adjust the length of generated text
        num_return_sequences=1,  # Number of different text sequences to generate
        no_repeat_ngram_size=2,  # Prevent repeating n-grams
        temperature=0.7,  # Controls the randomness of the output (higher = more creative)
        top_p=0.9,  # Nucleus sampling to consider only top-p probability mass
        top_k=50,  # Consider only top-k most probable tokens
        do_sample=True  # Enable sampling (instead of greedy decoding)
    )

    # Decode and print the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Test with a prompt
prompt = "In the future, artificial intelligence will"
generated_text = generate_text(prompt)

print("Generated Text:\n", generated_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Text:
 In the future, artificial intelligence will become the leading field of research in the field. It is going to be a key player in AI in fields like robotics, machine learning, and artificial neural networks.

This is an interesting time to take a look at the potential of AI to create new and interesting problems in human life. We know that there is a lot of work that needs to happen before AI can become an integral part of human society. The future of artificial life may be very different than


# Pipeline

In [26]:
# Function to perform OCR using Azure Computer Vision
def azure_ocr(image_path):
    with open(image_path, "rb") as image_stream:
        ocr_result = computervision_client.read_in_stream(image_stream, raw=True)

    # Get operation location (URL with an ID at the end) from response
    operation_location = ocr_result.headers["Operation-Location"]
    operation_id = operation_location.split("/")[-1]

    # Wait for the result
    while True:
        result = computervision_client.get_read_result(operation_id)
        if result.status not in [OperationStatusCodes.not_started, OperationStatusCodes.running]:
            break
    if result.status == OperationStatusCodes.succeeded:
        return result.analyze_result.read_results
    else:
        return None

In [27]:
# Function to extract text from OCR results
def extract_text(ocr_results):
    extracted_text = []
    for page in ocr_results:
        for line in page.lines:
            extracted_text.append(line.text)
    return " ".join(extracted_text)

In [32]:
# Function to use Hugging Face model to enhance text by filling in gaps
def enhance_text_with_genai(context_text):
    # Load the pre-trained model and tokenizer from Hugging Face
    model_name = "gpt2"  # You can also use "EleutherAI/gpt-neo-125M" or other models
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Tokenize the context text and generate predictions
    inputs = tokenizer.encode(context_text, return_tensors="pt")
    outputs = model.generate(inputs, max_length=1000, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)

    # Decode the generated text and return it
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [41]:
# Function to save color-coded text to a Word document
def save_color_coded_text_to_docx(ocr_text, enhanced_text, output_path="enhanced_text.docx"):
    ocr_words = ocr_text.split()
    enhanced_words = enhanced_text.split()

    # Create a new Document
    doc = Document()

    # Initialize the color-coded result
    i, j = 0, 0  # Pointers for ocr_words and enhanced_words
    while i < len(ocr_words) and j < len(enhanced_words):
        if ocr_words[i].lower() == enhanced_words[j].lower():  # Same word (OCR match)
            run = doc.add_paragraph().add_run(ocr_words[i])
            run.font.color.rgb = RGBColor(0, 255, 0)  # Green
            i += 1
            j += 1
        else:  # Word is from the generated text (no match)
            run = doc.add_paragraph().add_run(enhanced_words[j])
            run.font.color.rgb = RGBColor(0, 0, 255)  # Blue
            j += 1

    # If there are remaining words in OCR text (not covered by generated text)
    while i < len(ocr_words):
        run = doc.add_paragraph().add_run(ocr_words[i])
        run.font.color.rgb = RGBColor(0, 255, 0)  # Green
        i += 1

    # If there are remaining words in generated text
    while j < len(enhanced_words):
        run = doc.add_paragraph().add_run(enhanced_words[j])
        run.font.color.rgb = RGBColor(0, 0, 255)  # Blue
        j += 1

    # Save the document
    doc.save(output_path)
    print(f"Color-coded text saved to {output_path}")

In [42]:
# Full Workflow
def process_image(image_path):
    # Step 1: Perform OCR
    print("Performing OCR...")
    ocr_results = azure_ocr(image_path)
    if ocr_results:
        ocr_text = extract_text(ocr_results)
        print("OCR Text:", ocr_text)
    else:
        ocr_text = ""

    # Step 2: Enhance OCR results with Gen AI
    print("Enhancing OCR results with Gen AI...")
    enhanced_text = enhance_text_with_genai(ocr_text)
    print("Enhanced Text:", enhanced_text)

    # Step 3: Save the color-coded text to a Word document
    save_color_coded_text_to_docx(ocr_text, enhanced_text, output_path="enhanced_text.docx")

# Running the Pipeline

In [37]:
# Test the pipeline with an example image
image_path = "test_ocr.jpg"
enhanced_result = process_image(image_path)

Performing OCR...
OCR Text: Nasdaq & AMEX Stocks in bold rose or fell 5% or more USA Track your investments with our continuously TODAY updated stocks. Visit us on the web at .com money.usatoday.com 52-week 52-week High Low Stock Last Change High Low Stock Last Change 45.71 32.50 Biomet -A- 36.71 -0.42 2.76 1.20 Biomira 1.46 +0.03 9.07 5.13 BloScrip 8.05 +0.34 9.19 6.89 ABX Air n 7.52 -0.10 68.88 50.65 Biosite 50.05 -4.57 33.25 12.40 ACMoore 13.58 -1.57 212.25 131.03 BiotechT 204.66 -0.84 31.38 13.51 ADA-ES 20.96 +3.16 8.50 1.40 BirchMt gn 6.52 -0.45 27.14 12.88 ADC Tel rs 23.21 +0.13 18.21 10.73 Bickbaud 17.90 +0.70 30.40 16.70 ADECD 27.32 +0.73 52.73 13.86 BluCoat 41.29 +1.30 16.45 10.47 AFC Ent s 15.40 -0.14 44.35 24.15 BlueNile 40.30 -1.10 8.37 4.50 ASE Tst 7.76 +0.40 26.45 19.91 BobEvn 22.99 19.25 12.75 ASM Intl 17.65 -0.03 15.94 6.12 Bodisen n 15.45 +0.45 20.92 13.94 ASML Hid 21.24 +0.46 6.21 1.56 Bookham 5.94 +0.06 27.38 16.39 ASV Inc 5 26.76 +0.14 11.80 4.99 Borland 6.68 +0.14 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Enhanced Text: Nasdaq & AMEX Stocks in bold rose or fell 5% or more USA Track your investments with our continuously TODAY updated stocks. Visit us on the web at .com money.usatoday.com 52-week 52-week High Low Stock Last Change High Low Stock Last Change 45.71 32.50 Biomet -A- 36.71 -0.42 2.76 1.20 Biomira 1.46 +0.03 9.07 5.13 BloScrip 8.05 +0.34 9.19 6.89 ABX Air n 7.52 -0.10 68.88 50.65 Biosite 50.05 -4.57 33.25 12.40 ACMoore 13.58 -1.57 212.25 131.03 BiotechT 204.66 -0.84 31.38 13.51 ADA-ES 20.96 +3.16 8.50 1.40 BirchMt gn 6.52 -0.45 27.14 12.88 ADC Tel rs 23.21 +0.13 18.21 10.73 Bickbaud 17.90 +0.70 30.40 16.70 ADECD 27.32 +0.73 52.73 13.86 BluCoat 41.29 +1.30 16.45 10.47 AFC Ent s 15.40 -0.14 44.35 24.15 BlueNile 40.30 -1.10 8.37 4.50 ASE Tst 7.76 +0.40 26.45 19.91 BobEvn 22.99 19.25 12.75 ASM Intl 17.65 -0.03 15.94 6.12 Bodisen n 15.45 +0.45 20.92 13.94 ASML Hid 21.24 +0.46 6.21 1.56 Bookham 5.94 +0.06 27.38 16.39 ASV Inc 5 26.76 +0.14 11.80 4.99 Borland 6.68 +0.14 19.82 10.47 A

In [43]:
# Test the pipeline with an example image
image_path = "test_ocr2.jpg"
enhanced_result = process_image(image_path)

Performing OCR...
OCR Text: published weekly by HACHETTE FILIPACCHI ASSOCIES, 151, rue PERRET Cedex France. US Agent : USA Bound co ., 348 route 11, PO Box 771, Champlain, N.Y. 12919- 0771 USA. Subscription price is U.S. $ 149 per year. Periodicals postage paid at Champlain and at additional
Enhancing OCR results with Gen AI...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Enhanced Text: published weekly by HACHETTE FILIPACCHI ASSOCIES, 151, rue PERRET Cedex France. US Agent : USA Bound co ., 348 route 11, PO Box 771, Champlain, N.Y. 12919- 0771 USA. Subscription price is U.S. $ 149 per year. Periodicals postage paid at Champlain and at additional cost to the United States.

The following is a list of the most popular and popular books in the world. The list is not exhaustive. If you have any questions, please contact us.
Color-coded text saved to enhanced_text.docx


# template matching pipeline

In [46]:
!pip install opencv-python-headless numpy

In [47]:
import cv2
import numpy as np
from google.colab import files

In [48]:
# Function to perform template matching
def match_template(template_path, image_paths):
    # Load the template image
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    if template is None:
        print("Error: Template image not found.")
        return

    # Store the results
    results = []

    # Loop over the images to compare with the template
    for image_path in image_paths:
        # Load the image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Error: Image '{image_path}' not found.")
            continue

        # Perform template matching
        result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

        # Define a threshold for the match (this can be adjusted)
        threshold = 0.8

        if max_val >= threshold:
            results.append((image_path, "Match"))
        else:
            results.append((image_path, "No Match"))

    return results

In [49]:
# Function to display the results
def display_results(results):
    print("Image Matching Results:")
    for image_path, result in results:
        print(f"Image: {image_path} - {result}")

In [50]:
# Function to upload files to Colab
def upload_files():
    uploaded = files.upload()
    uploaded_files = list(uploaded.keys())
    return uploaded_files

In [51]:
# Main function
def main():
    # Upload the template image
    print("Upload the template image:")
    template_files = upload_files()
    template_path = template_files[0]  # Get the first uploaded file as the template

    print(f"Template image uploaded: {template_path}")

    # Upload the images to compare
    print("Upload the images to compare (one at a time, type 'done' when finished):")
    image_paths = []
    while True:
        uploaded_files = upload_files()
        if not uploaded_files:
            break
        for uploaded_file in uploaded_files:
            if uploaded_file.lower() == 'done':
                break
            image_paths.append(uploaded_file)

    print(f"Images to compare: {image_paths}")

    # Call the match_template function to process the images
    results = match_template(template_path, image_paths)

    # Display the results
    display_results(results)

In [ ]:
if __name__ == "__main__":
    main()

# Bloopers

# test OpenAI api

In [ ]:
from openai import AsyncOpenAI

# create client here

# Make an async request (ensure you are in an async function)
completion = await client.completions.create(
    model="gpt-4o",
    prompt="Hello World!",
    max_tokens=50
)

# Print the response text
print(completion.choices[0].text.strip())


# New Jankier method

In [ ]:
!pip install pyautogui

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.2/171.2 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyautogui: filename=PyAutoGUI-0.9.54-py3-none-any.whl size=37574 sha256=7a9735b7e63267c935f2c76f2092f40fdd0b56f16ae505bc17b1d

In [ ]:
import pyautogui
import time
from transformers import pipeline
from google.colab import files
from PIL import Image
import pyperclip  # To capture clipboard content
import io

# Load a text enhancement model from HuggingFace
text_enhancer = pipeline("text2text-generation", model="t5-base")

KeyError: 'DISPLAY'

In [ ]:
# Function to upload an image in Colab
def upload_image():
    uploaded = files.upload()
    image_path = next(iter(uploaded))  # Extract the first uploaded file name
    return image_path

In [ ]:
# Function to display image in Colab
def display_image(image_path):
    img = Image.open(image_path)
    img.show()

In [ ]:
# Trigger PowerToys Text Extractor (run on local machine)
def trigger_powertoys_text_extractor():
    # This part of the script should be run on your local machine (Windows)
    # Simulate the hotkey to trigger PowerToys' Text Extractor
    pyautogui.hotkey('ctrl', 'shift', 't')  # Default hotkey for Text Extractor
    time.sleep(1)
    pyautogui.click(500, 500)  # Adjust click position
    time.sleep(2)  # Allow PowerToys time to perform OCR

In [ ]:
# Function to capture text from clipboard (after OCR in PowerToys)
def capture_text_from_clipboard():
    # Captures text copied to clipboard by PowerToys
    extracted_text = pyperclip.paste()
    return extracted_text

In [ ]:
# Function to enhance the extracted text
def enhance_text(text):
    enhanced_text = text_enhancer(text)[0]['generated_text']
    return enhanced_text

In [ ]:
def run_pipeline():
    # Step 1: Upload the image to Colab
    image_path = upload_image()

    # Step 2: Display the image in Colab for visual confirmation
    display_image(image_path)

    # Step 3: Trigger PowerToys Text Extractor (run this on your local machine)
    print("Please run 'trigger_powertoys_text_extractor()' on your local machine.")

    # Step 4: Capture the extracted text from clipboard (from PowerToys)
    extracted_text = capture_text_from_clipboard()

    # Step 5: Enhance the extracted text
    enhanced_text = enhance_text(extracted_text)

    print("Original Extracted Text: ", extracted_text)
    print("Enhanced Text: ", enhanced_text)

In [ ]:
# Run the pipeline
run_pipeline()